***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-fb40  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 10 17:38 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'anchor_bucket_1'
 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)
        
    

# Building an inverted index


Reading the entire corpus and partitioning it to 3 parts - body, anchor and title.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.limit.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

Importing the inverted index module.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import *

**Helper functions**

In [ ]:
# define the stopwords
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)

# tokenize and stemming the words if defined
def rearrange_words(words_txt, stemm=False):
    set_of_words = []
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
    Port_stem = PorterStemmer()
    words = [word.group() for word in RE_WORD.finditer(words_txt.lower()) if word.group() not in all_stopwords]
    if stemm:
      Port_stem = PorterStemmer()
      for t in words:
          if t not in all_stopwords: 
              set_of_words.append(Port_stem.stem(t))
      return set_of_words
    return words


In [ ]:
def word_count(text, id, stemm=False):
  '''Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = rearrange_words(text)
  # Dictionary of tokens:
  terms_posting={}
  stemmer = PorterStemmer()  
  # For each token:
  for ind in tokens:
    # This is not in the removal words:
    if ind not in all_stopwords:
      if stemm:
        ind = stemmer.stem(ind)  
      # If you saw this word before:
      if ind in terms_posting:
        # Add to this doc_id or new doc_id (tf+1):
        terms_posting[ind]=(id,terms_posting[ind][1]+1)
      else:
        # Not seen before put the doc_id with (tf=1):
        terms_posting[ind]=(id,1)
  res = list(terms_posting.items())
  return (res)

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # Just sort the tuples: Shuffling:
  return(sorted(unsorted_pl))

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # How many docs include in them each token:
  return (postings.mapValues(lambda tokens: len(tokens)))


In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,buck_n):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # Processes the data by bucket id and pair for each of the pairs
  # according to the given function:
  mapper = postings.map(lambda pair: (token2bucket_id(pair[0]),pair))
  # Merge all the records by the buckets id:
  reducer = mapper.groupByKey()
  # Need to write each of the buckets to the disk:
  w_block = reducer.map(lambda buck:InvertedIndex.write_a_posting_list(buck,buck_n))
  return (w_block)

In [ ]:
# Create the posting lists, writing them into the bucket and calculating the document frequency for each word.

def posting_list_pairs(doc_par,part_text, stemm=False):
    word_counts = doc_par.flatMap(lambda x: word_count(x[0], x[1], stemm)) # -> Vw: [ (w,(doc_id,tf)), ...]
    list_post = word_counts.groupByKey().mapValues(reduce_word_counts) # -> Vw:[ (w,[(doc_id,tf), ...]),....]
    if(part_text == 'body'):
        list_post = list_post.filter(lambda x: len(x[1])>50) # filter out rare words, words that appear in 50 or fewer documents 
    
    word2df_cal = calculate_df(list_post).collectAsMap() # -> Vw: {w : DF , ....}
    _ = partition_postings_and_write(list_post,f"{part_text}_bucket_1").collect() # write to bucket (w,df)
    return list_post, word2df_cal

In [ ]:
#Manipulation on anchor text
doc_anchor = doc_anchor_pairs.map(lambda x:(" ".join([i[1] for i in x[0]]),x[1]))

In [ ]:
client = storage.Client()
# collect all posting lists locations into one super-set
def super_posting_locs(buck_n):
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(buck_n, prefix='postings_gcp'):
      if not blob.name.endswith("pickle"):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
    return super_posting_locs

Putting it all together

In [ ]:
from math import sqrt
import numpy as np

def inv_idx(p_txt, stemm=False):
    # Create inverted index instance
    inv = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    if p_txt == "body":
        # Extract df and posting list for every word in the body
        post_fil, inv.df = posting_list_pairs(doc_text_pairs,p_txt, stemm)
         # Corpus length 
        inv.corpus_len = parquetFile.count()
        # Calculating the idf for each word in the body
        inv.idf_dict = {word:np.log10(inv.corpus_len/val) for word, val in inv.df.items()} # {word : idf_value, ...}
        # Tokenize the word in the body
        body_docs_tokenized = doc_text_pairs.map(lambda x: (x[1], rearrange_words(x[0], stemm))) # [(doc_id,[token1,..., token n]),...]
        # Calculating the length of each document
        DL_map = body_docs_tokenized.map(lambda x: (x[0], len(x[1])))
        inv.DL = DL_map.collectAsMap()

    elif p_txt == "title":
        # Extract df and posting list for every word in the title
        post_fil, inv.df = posting_list_pairs(doc_title_pairs,p_txt, stemm)
        # Extract titles
        inv.title = doc_title_pairs.toDF().select("id","title").rdd.collectAsMap() 
        # Tokenize the word in the title       
        body_docs_tokenized = doc_title_pairs.map(lambda x: (x[1], rearrange_words(x[0], stemm))) # [(doc_id,[token1,..., token n]),...]
        # Calculating the length of each document
        DL_dict = {}
        for i in body_docs_tokenized.collect():
            DL_dict[i[0]] = len(i[1])
        inv.DL = DL_dict #{ doc_id: doc_len, ...}
        # Corpus length 
        inv.corpus_len = parquetFile.count() 
               
    elif p_txt == "anchor":
        # Extract df and posting list for every word in the anchor
        post_fil, inv.df = posting_list_pairs(doc_anchor,p_txt, stemm)
        # Tokenize the word in the anchor
        body_docs_tokenized = doc_anchor.map(lambda x: (x[1], rearrange_words(x[0], stemm))) # [(doc_id,[token1,..., token n]),...]
        # Calculating the length of each document
        DL_map = body_docs_tokenized.map(lambda x: (x[0], len(x[1])))
        inv.DL = DL_map.collectAsMap()
        # Corpus length
        inv.corpus_len = parquetFile.count()  
        
    # Adding the posting locations dictionary to the inverted index
    inv.posting_locs = super_posting_locs(f"{p_txt}_bucket_1")
    # write the global stats out
    inv.write_index('.', f'index_{p_txt}_stemming_{stemm}')
    # upload to gs
    index_src = f"index_{p_txt}_stemming_{stemm}.pkl"
    index_dst = f'gs://{p_txt}_bucket_1/postings_gcp_stemming_{stemm}/{index_src}'
    !gsutil cp $index_src $index_dst

**Creating the inverted index without stemming**

In [ ]:
inv_idx("title")
inv_idx("body")
inv_idx("anchor")

**Creating the inverted index with stemming**

In [ ]:
inv_idx("title", stemm=True)
inv_idx("body", stemm=True)
inv_idx("anchor", stemm=True)

## PageRank:

Compute PageRank for the entire English Wikipedia.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # Calculate edges by the raws of the pages:
  ed = pages.flatMap(lambda raw:[(raw[0],tup[0]) for tup in raw[1]])
  edges = ed.distinct()
  # Calculate vertices by the tuples of edges:
  ver = edges.flatMap(lambda tup:[(ind,) for ind in tup])
  vertices = ver.distinct()
  return edges, vertices

In [ ]:
pages_links = parquetFile.select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr = (pr.toPandas()).to_dict()
pr = pd.DataFrame(pr)
pr.to_pickle(f'gs://{bucket_name}/pr/page_rank.pkl')

##  Page Views:

In [ ]:
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'

# Download the file (2.3GB): 
!wget -N $pv_path

# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
